In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

# Functions

In [2]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [3]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,method
0,0.116263,exp1,Sphere,1.0,NN_HMu
1,0.116340,exp1,Sphere,1.0,NN_HMu
2,0.116470,exp1,Sphere,1.0,NN_HMu
3,0.115846,exp1,Sphere,1.0,NN_HMu
4,0.115519,exp1,Sphere,1.0,NN_HMu


In [4]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])

In [5]:
print(table2.to_csv(float_format='%.2f'))

experiment,,exp1,exp1,exp1,exp2,exp2,exp2,exp3,exp3,exp3,exp4,exp4,exp4
freq,,0.5,1.0,4.0,0.5,1.0,4.0,0.5,1.0,4.0,0.5,1.0,4.0
function,method,,,,,,,,,,,,
Rastrigin,NN_CwN,0.24,0.12,0.03,0.24,0.10,0.03,0.23,0.23,0.03,0.23,0.12,0.03
Rastrigin,NN_HMu,0.19,0.11,0.03,0.20,0.12,0.03,0.21,0.12,0.03,0.20,0.10,0.03
Rastrigin,NN_No,0.22,0.23,0.03,0.19,0.14,0.03,0.47,0.13,0.02,0.19,0.10,0.03
Rastrigin,NN_RI,0.23,0.12,0.03,0.23,0.13,0.03,0.24,0.12,0.03,0.23,0.12,0.02
Rosenbrock,NN_CwN,0.24,0.11,0.03,0.54,0.12,0.03,0.23,0.10,0.03,0.23,0.11,0.03
Rosenbrock,NN_HMu,0.19,0.11,0.03,0.20,0.11,0.03,0.20,0.12,0.03,0.20,0.10,0.03
Rosenbrock,NN_No,0.22,0.23,0.03,0.22,0.12,0.03,0.47,0.13,0.02,0.19,0.10,0.02
Rosenbrock,NN_RI,0.25,0.12,0.03,0.24,0.11,0.02,0.24,0.12,0.03,0.23,0.11,0.03
Sphere,NN_CwN,0.20,0.11,0.06,0.23,0.11,0.03,0.23,0.10,0.03,0.22,0.11,0.03
Sphere,NN_HMu,0.19,0.12,0.08,0.19,0.11,0.03,0.20,0.12,0.03,0.20,0.10,0.02
Sphere,NN_No,0.19,0.22,0.03,0.23,0.12,0.03,0.19,0.15,0.02,0.20,0.10,0.02
Sphere,NN

In [6]:
table2

experiment             exp1                          exp2                      \
freq                    0.5       1.0       4.0       0.5       1.0       4.0   
function   method                                                               
Rastrigin  NN_CwN  0.238858  0.117118  0.025614  0.235390  0.096084  0.029276   
           NN_HMu  0.194400  0.109923  0.030029  0.197391  0.120002  0.030040   
           NN_No   0.219638  0.232658  0.030587  0.194311  0.144632  0.025029   
           NN_RI   0.233593  0.121731  0.029455  0.230982  0.128001  0.031310   
Rosenbrock NN_CwN  0.240905  0.110085  0.025005  0.538075  0.117893  0.029078   
           NN_HMu  0.191618  0.112139  0.028908  0.197156  0.113104  0.029592   
           NN_No   0.216815  0.231278  0.031025  0.222852  0.118871  0.029030   
           NN_RI   0.246094  0.118646  0.028376  0.242696  0.111867  0.023240   
Sphere     NN_CwN  0.195864  0.109161  0.057282  0.225201  0.111110  0.028285   
           NN_HMu  0.193904  0.116209  0.076557  0.193630  0.111979  0.029520   
           NN_No   0.190512  0.224787  0.027131  0.231055  0.118152  0.029321   
           NN_RI   0.249047  0.119492  0.028112  0.246664  0.126085  0.025838   

experiment             exp3                          exp4                      
freq                    0.5       1.0       4.0       0.5       1.0       4.0  
function   method                                                              
Rastrigin  NN_CwN  0.227748  0.233631  0.025715  0.231268  0.117296  0.028107  
           NN_HMu  0.211723  0.118693  0.029274  0.195174  0.098611  0.026449  
           NN_No   0.468137  0.131614  0.024570  0.194444  0.098360  0.029155  
           NN_RI   0.244591  0.118132  0.025679  0.227149  0.117316  0.024924  
Rosenbrock NN_CwN  0.229555  0.100271  0.025545  0.228012  0.114862  0.028964  
           NN_HMu  0.202382  0.119272  0.029639  0.196466  0.099299  0.027550  
           NN_No   0.468608  0.128810  0.024055  0.194706  0.097970  0.024597  
           NN_RI   0.242620  0.123575  0.025452  0.230109  0.111896  0.025408  
Sphere     NN_CwN  0.234513  0.097867  0.029458  0.224315  0.112368  0.028224  
           NN_HMu  0.200457  0.115876  0.029944  0.203246  0.098770  0.024798  
           NN_No   0.194656  0.146022  0.024925  0.195494  0.096987  0.024392  
           NN_RI   0.238167  0.123777  0.027440  0.237559  0.117667  0.031197